In [1]:
from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io


matlab_data1 = scipy.io.loadmat('../matlab/thermal_data1.mat',mat_dtype=False)  # is set to True, givex complex casting to real errors..
matlab_data2 = scipy.io.loadmat('../matlab/thermal_data3.mat',mat_dtype=False)  # is set to True, givex complex casting to real errors..

In [2]:
beta1 = matlab_data1["beta"][0,0]
beta1 = float(beta1)
print('beta1 is ' ,beta1)

beta2 = matlab_data2["beta"][0,0]
beta2 = float(beta2)
print('beta2 is ' ,beta2)


w0list = matlab_data2["w0list"][:,0]
w0list = w0list.astype('float')
print('w0list is ', w0list)

glist = matlab_data1["glist"][:,0]
glist = glist.astype('float')

print('glist is', glist)

NL = int(matlab_data1["NL"][0,0])
NM = int(matlab_data2["NM"][0,0])

print('NL and NM are ', NL, NM, 'respectively')

N = NL + NM
dL = 2**NL
dM = 2**NM
d = 2**N
dims = [2]*N

# Parameters can be modified here for testing purposes. 

beta1 is  1.0
beta2 is  2.3696682464454977
w0list is  [1. 1. 1. 1. 1. 1.]
glist is [0.1 0.1 0.1 0.1 0.1]
NL and NM are  2 4 respectively


In [3]:
H_S = create_hamiltonian2(w0list,glist,N)
eigenenergies, eigenstates = H_S.eigenstates()
number = len(eigenenergies) # should be 2^N

print(eigenstates)

[Quantum object: dims=[[2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1]], shape=(64, 1), type='ket', dtype=Dense
 Qobj data =
 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]]
 Quantum object: dims=[[2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1]], shape=(64, 1), type='ket', dtype=Dense
 Qobj data =
 [[ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
  [ 1.34080119e-16]
  [ 0.00000000e+00]
  [-6.69102454e-17]
  [-4.37029292e-17]
  [ 2.52805984e-48]
  [ 5.51166031e-47]
  [-3.65350309e-18]
  [ 1.18204294e-16]
  [ 1.20238972e-43]
  [ 1.56663624e-16]
  [ 2.59875560e-17]
  [ 1.75041507e-16]
  [-3.53980855e-19]

In [4]:
M = matlab_data2["gamma_matrix"][0,0]

val1 = M
final_val = np.asarray(val1)
print(final_val)

#np.linalg.norm(val1)

(3.0993677507138146e-07+0j)


In [5]:
final_val2 = np.array(final_val)
print(final_val2)

(3.0993677507138146e-07+0j)


In [6]:
matlab_F_list1 = matlab_data1["F"]
F_list1 = []

matlab_F_list2 = matlab_data2["F"]
F_list2 = []


for index in range(2**dL):
    matrix1 = np.asmatrix(matlab_F_list1[0,index])
    matrix2 = np.asmatrix(matlab_F_list2[0,index])

    F_list1.append(Qobj(matrix1))
    F_list2.append(Qobj(matrix2))
    
if not basis_is_orthonormal(F_list1):
    print("WARNING : Basis 1 is NOT orthonormal")

if not basis_is_orthonormal(F_list2):
    print("WARNING : Basis 2 is NOT orthonormal")

F_list1.pop() #removes the last indentity basis ..
print(F_list2[-1]) # should be identity matrix
F_list2.pop()

for index in range(2**dL-1):
    F_list1[index].dims = [dims,dims] # makes sure mutliplication will work and QuTiP wont throw errors. 
    F_list2[index].dims = [dims,dims]

final_val1 = np.asarray(matlab_data1["gamma_matrix"])
final_val2 = np.asarray(matlab_data2["gamma_matrix"])


Quantum object: dims=[[64], [64]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.125 0.    0.    ... 0.    0.    0.   ]
 [0.    0.125 0.    ... 0.    0.    0.   ]
 [0.    0.    0.125 ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.125 0.    0.   ]
 [0.    0.    0.    ... 0.    0.125 0.   ]
 [0.    0.    0.    ... 0.    0.    0.125]]


In [7]:
gamma_matrix1 = np.zeros((dL**2-1,dL**2-1), dtype = complex)
gamma_matrix2 = np.zeros((dL**2-1,dL**2-1), dtype = complex)

for i in range(dL**2-1):
    for j in range(dL**2-1):
        gamma_matrix1[i,j] = matlab_data1["gamma_matrix"][i,j]
        gamma_matrix2[i,j] = matlab_data2["gamma_matrix"][i,j]

In [8]:
print(gamma_matrix1)
print(gamma_matrix2)

#print(gamma_matrix1[0,0])

[[ 2.24519462e-07+0.00000000e+00j  4.05134719e-16+1.42281248e-33j
  -8.63594179e-15+1.00836256e-33j -5.53035183e-06+1.37060556e-30j
   1.54122434e-18-5.22914759e-36j  0.00000000e+00+0.00000000e+00j
  -1.33063872e-07+3.14223401e-32j  1.49879179e-14+6.34413361e-32j
  -8.99029551e-17+7.53556164e-36j -8.48418177e-07-3.49346283e-32j
   0.00000000e+00+0.00000000e+00j  3.02643677e-13-3.35578246e-32j
  -8.95846339e-08-6.97875384e-34j  2.38879236e-17+2.52380106e-34j
   2.17767485e-15-2.61475494e-34j]
 [ 4.05134719e-16-1.42281248e-33j  4.76395616e-04+0.00000000e+00j
   1.80493037e-12-4.27843833e-31j -4.42602070e-12+2.16609341e-30j
   1.05269901e-06-1.12621199e-31j  0.00000000e+00+0.00000000e+00j
  -8.98738257e-14+9.82521437e-32j  1.70049570e-02-3.09218067e-28j
   1.65392450e-14-5.02994033e-33j  1.58560842e-13+6.45414107e-32j
   0.00000000e+00+0.00000000e+00j -6.66057241e-11+1.47288381e-29j
   2.96962572e-15+1.24876291e-33j -1.87223589e-05-9.41651059e-30j
  -4.15189969e-13+1.32855688e-31j]
 [-8.6

In [10]:
print(np.trace(gamma_matrix1))

(1.0000000000700926+0j)


In [24]:
[D1,U1] = np.linalg.eigh(gamma_matrix1)
[D2,U2] = np.linalg.eigh(gamma_matrix2)

print(D1)
print(D2)

[9.81958033e-08 1.38883916e-07 2.66966530e-07 3.77855835e-07
 6.57514963e-06 9.51556830e-06 1.25815261e-05 1.84264694e-05
 2.80327173e-05 3.18979664e-05 4.62157785e-05 1.01570238e-04
 6.45641744e-02 2.51674501e-01 6.83505627e-01]
[8.68861020e-08 1.58758603e-07 6.42428637e-07 1.19335683e-06
 5.90155909e-06 8.47289192e-06 2.22641687e-05 4.40338231e-05
 5.99517744e-05 1.37280607e-04 2.57833572e-04 3.62884556e-04
 1.36646496e-03 1.19472511e-01 8.78260319e-01]


Note that we have gotten now $ U^\dagger \Gamma U = D$ where $U$ is a diagonal matrix. Therefore the Dissipator term is written as
$$
D(\rho) = \sum_{\alpha, \beta}  \Gamma_{\alpha,\beta} (F_\beta \rho F_\alpha^\dagger - \{ F_\alpha^\dagger F_\beta, \rho\}/2 ) \\
= \sum_{\alpha, \beta} \sum_{i,j} (U_{\alpha,i} D_{i,j} U^\dagger_{j,\beta})  (F_\beta \rho F_\alpha^\dagger - \{ F_\alpha^\dagger F_\beta, \rho\}/2 )  \\
 = \sum_{i,j}  D_{i,j}   (L_j \rho L_i^\dagger - \{ L_i^\dagger L_j, \rho\}/2 ) $$

where $L_j = \sum_{\beta} U^\dagger_{j,\beta} F_\beta $
We can define $\tilde{L}_j = \sqrt{D_{jj}} L_j$ to get
$$ D(\rho) = \sum_j ( \tilde{L}_j \rho \tilde{L}_j^\dagger - \{ \tilde{L}_j^\dagger \tilde{L}_j, \rho\}/2 ) $$

In [25]:
## Constructing the lindblad operators we have

L_list1 = []
Udagger1 = np.transpose(np.conj(U1))
L_list2 = []
Udagger2 = np.transpose(np.conj(U2))

for j  in range(dL**2-1):
    op1 = 0
    op2 = 0
    for betaindex in range(dL**2-1):    
        op1 = op1+Udagger1[j,betaindex]* F_list1[betaindex]
        op2 = op2+Udagger2[j,betaindex]* F_list2[betaindex]
    L_list1.append(op1)
    L_list2.append(op2)

tildeL_list1 = []
tildeL_list2 = []

for index in range(dL**2-1):
    tildeL_list1.append(np.sqrt(D1[index])*L_list1[index])
    tildeL_list2.append(np.sqrt(D2[index])*L_list2[index])

In [26]:


rho_gibbs1 = (-beta1*H_S).expm() 
rho_gibbs2 = (-beta2*H_S).expm()
rho_th1 = rho_gibbs1/rho_gibbs1.tr()
rho_th2 = rho_gibbs2/rho_gibbs2.tr()


rhodot_thermal = 0

for i in range(dL**2-1):
    L1 = tildeL_list1[i]
    L2 = tildeL_list2[i]
    rhodot_thermal = rhodot_thermal+ L1*rho_th1*L1.dag() - 0.5*L1.dag()*L1*rho_th1 - 0.5*rho_th1*L1.dag()*L1 + L2*rho_th2*L2.dag() - 0.5*L2.dag()*L2*rho_th2 - 0.5*rho_th2*L2.dag()*L2

violation  = np.zeros((d,1),dtype = complex)
for i in range(d):
    violation[i] = (eigenstates[i].dag()*rhodot_thermal*eigenstates[i]) #affected by ordering of U

tildeL_list = []
for i in tildeL_list1:
    tildeL_list.append(i)

for i in tildeL_list2:
    tildeL_list.append(i)

print(len(tildeL_list))


rhodot_thermal_v2 = 0

for  alphaindex in range(dL**2-1):
    for betaindex in range(dL**2-1):
        Falpha1 = F_list1[alphaindex]
        Fbeta1 = F_list1[betaindex]

        Falpha2 = F_list2[alphaindex]
        Fbeta2 = F_list2[betaindex]
       
        
       
        rhodot_therma_v2 = rhodot_thermal_v2 + gamma_matrix1[alphaindex,betaindex]*(Fbeta1*rho_th1*Falpha1.dag()-0.5*Falpha1.dag()*Fbeta1*rho_th1 - 0.5*rho_th1*Falpha1.dag()*Fbeta1) + gamma_matrix2[alphaindex,betaindex]*(Fbeta2*rho_th2*Falpha2.dag()-0.5*Falpha2.dag()*Fbeta2*rho_th2 - 0.5*rho_th2*Falpha2.dag()*Fbeta2)


violation_v2  = np.zeros((d,1),dtype = complex)
for i in range(d):
    violation_v2[i] = (eigenstates[i].dag()*rhodot_thermal_v2*eigenstates[i]) #affected by ordering of U

30


In [28]:
H_LS_matrix1 = matlab_data1["H_LS"]
H_LS_matrix2 = matlab_data2["H_LS"]
H_LS1 = tensor(Qobj(H_LS_matrix1),qeye(dM))
H_LS2 = tensor(qeye(dM),Qobj(H_LS_matrix2))
H_LS1.dims = [dims,dims]
H_LS2.dims = [dims,dims]
rho_ss = steadystate(H_S + H_LS1 + H_LS2, tildeL_list)

trace_dist = tracedist(rho_ss,rho_th1)


L_operator = liouvillian(H_S+H_LS1+H_LS2, tildeL_list)

L_eigen = L_operator.eigenenergies()

In [30]:
L_v2 = spre(-1.0j*(H_S+H_LS1 + H_LS2)) + spost(1.0j*(H_S+H_LS1 + H_LS2))

for alphaindex in range(dL**2-1):
    for betaindex in range(dL**2-1):
        Falpha1 = F_list1[alphaindex]
        Fbeta1 = F_list1[betaindex]

        Falpha2 = F_list2[alphaindex]
        Fbeta2 = F_list2[betaindex]
        L_v2= L_v2+gamma_matrix1[alphaindex,betaindex]*(spre(Fbeta1)*spost(Falpha1.dag()) - 0.5*spre(Falpha1.dag()*Fbeta1) - 0.5*spost(Falpha1.dag()*Fbeta1)) + gamma_matrix2[alphaindex,betaindex]*(spre(Fbeta2)*spost(Falpha2.dag()) - 0.5*spre(Falpha2.dag()*Fbeta2) - 0.5*spost(Falpha2.dag()*Fbeta2))


rho_ss_v2 = steadystate(L_v2)
trace_dist_v2 = tracedist(rho_ss_v2,rho_th1)
L_eigen_v2 = L_v2.eigenenergies()

In [31]:
# print("violation is ",violation)
#print("violation_v2 is ",violation_v2)
# print()
#print("########################################################")
 #print()


print("Trace distance betweens steady state and thermal state is", trace_dist)

print("Smallest eigenvalues of L are ", L_eigen[-3:])

#print()
#print("########################################################")
#print()

print("Trace distance betweens steady state v2 and thermal state is", trace_dist_v2)

print("Smallest eigenvalues of L_v2 are ", L_eigen_v2[-3:])

Trace distance betweens steady state and thermal state is 0.1843922092954619
Smallest eigenvalues of L are  [-3.45744806e-03+3.60103429e-01j -3.16574462e-03+1.23026593e-16j
 -1.79047840e-16-1.08661395e-14j]
Trace distance betweens steady state v2 and thermal state is 0.1843922092954611
Smallest eigenvalues of L_v2 are  [-3.45744806e-03-3.60103429e-01j -3.16574462e-03+6.62035776e-17j
 -1.69562750e-15-2.11979850e-15j]


In [32]:
print(rho_ss_v2)
print(rho_ss)

print("Trace of rho_ss is ", rho_ss.tr())
print("Trace of rho_ss_v2 is ", rho_ss_v2.tr())

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 4.83693394e-05+0.00000000e+00j  4.06616948e-12+1.00655856e-10j
  -1.10661576e-11+5.04215178e-11j ... -3.97336391e-25-1.42770943e-25j
  -1.70745665e-26+2.16290225e-26j -4.23117701e-28+1.12973096e-28j]
 [ 4.06616948e-12-1.00655856e-10j  2.08869299e-04+0.00000000e+00j
  -2.89383784e-05+1.50023032e-05j ... -1.51597588e-21+3.45298007e-21j
  -2.07564357e-22-1.48622844e-22j -1.61656276e-27+3.42096256e-26j]
 [-1.10661576e-11-5.04215178e-11j -2.89383784e-05-1.50023032e-05j
   2.09318361e-04+0.00000000e+00j ... -3.18769635e-20-3.72845003e-20j
   2.41154833e-21+4.36839937e-21j -2.61125827e-25-5.48254845e-25j]
 ...
 [-3.97336391e-25+1.42770943e-25j -1.51597588e-21-3.45298007e-21j
  -3.18769635e-20+3.72845003e-20j ...  6.64658701e-02+0.00000000e+00j
  -9.06897124e-03+4.64348453e-03j  1.41481786e-09-2.09588219e-08j]
 [-1.70745665e-26-2.16290225e-26j -2.07564357e-22+1.48

In [ ]:
#Check if the steady state is a valid density matrix
print("Is rho_ss a valid density matrix? ", rho_ss.isherm)
print("Is rho_ss_v2 a valid density matrix? ", rho_ss_v2.isherm)

rho_ss

In [27]:
print(violation)
print(violation_v2)

[[-9.86083888e-12+9.33069253e-41j]
 [ 2.03898204e-12-1.84268209e-37j]
 [-3.64456550e-13-1.30974957e-37j]
 [-3.62991916e-12-8.13703772e-38j]
 [ 3.80167200e-12-9.99225726e-38j]
 [-1.56663241e-12-1.22804987e-37j]
 [-1.90929699e-12-1.57916062e-37j]
 [-5.21474338e-11+8.48189961e-39j]
 [-1.16821723e-10+3.20676475e-38j]
 [ 3.13979869e-11+3.39035906e-38j]
 [-5.03393790e-12+3.58577009e-38j]
 [ 1.76740572e-10+9.21454046e-39j]
 [-8.35527064e-12+2.52579063e-38j]
 [ 9.93227693e-12+4.91425519e-38j]
 [-6.04113597e-12+4.87354592e-38j]
 [-5.88055648e-12+5.81250893e-38j]
 [-2.86269497e-12+3.11773978e-38j]
 [-1.30290538e-10+1.75508937e-38j]
 [ 6.51418813e-12+4.01176049e-38j]
 [-6.33855500e-12+2.21410430e-38j]
 [ 7.90621815e-11+2.48310518e-38j]
 [ 2.70394650e-11+9.89424435e-39j]
 [-2.28071240e-11+8.12555513e-39j]
 [ 2.74997675e-11+5.53152993e-39j]
 [-1.05296073e-11+2.04060628e-38j]
 [-1.29904872e-11+8.19155917e-39j]
 [ 2.65743678e-11+8.76405260e-39j]
 [-1.33559332e-12+2.10794420e-38j]
 [-4.13163816e-11+1.

In [39]:
print(len(L_eigen_v2))
print(L_eigen_v2)

4096
[-4.42016348e-02+5.00000000e-01j -4.42016348e-02-5.00000000e-01j
 -4.42016348e-02+5.00000000e-01j ...  7.89940506e-16-3.89075341e-15j
  1.08420217e-15+5.00000000e-01j  2.07180193e-15-5.00000000e-01j]


In [10]:
H_test = create_projector01(2,1).dag()
print(H_test)

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]
